*Este script balancea el numero de imagenes en cada una de las 16 clases tanto del folder train como el de test.
Primero se recorren train y test para encontrar la clase con mayor numero de imagenes (mayor numero en train y en test respectivamente).
Una vez hecho esto se generan imagenes en cada una de los clases que tengan menos imagenes que la clase encontrada en el paso anterior.
Así al teminar de correr el script todas las clases tanto en train tienen el mismo numero de imagenes, y todas las clases en tes tienen el mismo numero de imagenes (el dataset esta dividido aproximadente en 80% train y 20% test).*

In [3]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import shutil
from PIL import Image, UnidentifiedImageError

In [4]:
# Se generan los paths de los directorios, y se obtienen las listas de los subdirectorios dentro de train y test
current_dir = os.getcwd()

dataset_path = os.path.join(current_dir, "dataset")
train_path = os.path.join(current_dir, "dataset/train")
test_path = os.path.join(current_dir, "dataset/test")

train_subfolders = [f.name for f in os.scandir(train_path) if f.is_dir()]

test_subfolders = [f.name for f in os.scandir(test_path) if f.is_dir()]

print("train subfolders: ", train_subfolders)
print("test subfolders: ", test_subfolders)

train subfolders:  ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'add', 'dec', 'div', 'eq', 'mul', 'sub', 'x', 'y', 'z']
test subfolders:  ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'add', 'dec', 'div', 'eq', 'mul', 'sub', 'x', 'y', 'z']


In [5]:
# Se encuentra la cantidad máxima de imágenes entre todas los subdirectorios de test y train
max_train_images = max([len(os.listdir(os.path.join(train_path, subfolder))) for subfolder in train_subfolders])
max_test_images = max([len(os.listdir(os.path.join(test_path, subfolder))) for subfolder in test_subfolders])

print("max train images: ", max_train_images)
print("max test images: ", max_test_images)

max train images:  524
max test images:  131


*Se configura ImageDataGenerator, las nuevas imagenes generadas solamente tendran variacion en el brillo y en el zoom
Se configuró de esta manera ya que al trabajar con simbolos matematicos el girar o cambiar la posicion de la imagen
afecta al dataset dedido a que haciendo esto se podria obtener un simbolo diferente, y ensuciaria los datos de la clase.*

In [6]:
data_generator = ImageDataGenerator(
    rescale=1./255,
    brightness_range=[0.8, 1.2],  # Adjusted brightness range
    zoom_range=[0.9, 1.1],  # Adjusted zoom range
    shear_range=0.0,  # Set shear range to 0
    horizontal_flip=False  # Disable horizontal flipping
)

In [7]:
# Se itera sobtre los subdirectorios de train y se balancea el numero de imagenes
for subfolder in train_subfolders:
    folder_path = os.path.join(train_path, subfolder)
    num_images = len(os.listdir(folder_path))
    if num_images < max_train_images:
        images = os.listdir(folder_path)
        num_generate = max_train_images - num_images
        for i in range(num_generate):
            img_path = os.path.join(folder_path, images[i % num_images])
            try:
                img = Image.open(img_path).convert("RGB")
                img = tf.keras.preprocessing.image.img_to_array(img)
                img = img.reshape((1,) + img.shape)
                generated_images = data_generator.flow(
                    img,
                    batch_size=1,
                    save_to_dir=folder_path,
                    save_prefix='generated_',
                    save_format='jpg'
                )
                for batch in generated_images:
                    break
            except (tf.errors.InvalidArgumentError, UnidentifiedImageError):
                continue



In [8]:
# Se itera sobtre los subdirectorios de test y se balancea el numero de imagenes
for subfolder in test_subfolders:
    folder_path = os.path.join(test_path, subfolder)
    num_images = len(os.listdir(folder_path))
    if num_images < max_test_images:
        images = os.listdir(folder_path)
        num_generate = max_test_images - num_images
        for i in range(num_generate):
            img_path = os.path.join(folder_path, images[i % num_images])
            try:
                img = Image.open(img_path).convert("RGB")
                img = tf.keras.preprocessing.image.img_to_array(img)
                img = img.reshape((1,) + img.shape)
                generated_images = data_generator.flow(
                    img,
                    batch_size=1,
                    save_to_dir=folder_path,
                    save_prefix='generated_',
                    save_format='jpg'
                )
                for batch in generated_images:
                    break
            except (tf.errors.InvalidArgumentError, UnidentifiedImageError):
                continue

In [9]:
# Imprime el numero de imagenes en cada folder de train
print("Number of images in each train folder:")
for subfolder in train_subfolders:
    folder_path = os.path.join(train_path, subfolder)
    num_images = len(os.listdir(folder_path))
    print(f"{subfolder}: {num_images} images")

Number of images in each train folder:
0: 524 images
1: 524 images
2: 524 images
3: 524 images
4: 524 images
5: 524 images
6: 524 images
7: 524 images
8: 524 images
9: 523 images
add: 524 images
dec: 524 images
div: 524 images
eq: 524 images
mul: 524 images
sub: 524 images
x: 524 images
y: 524 images
z: 524 images


In [10]:
# Imprime el numero de imagenes en cada folder de test
print("\nNumber of images in each test folder:")
for subfolder in test_subfolders:
    folder_path = os.path.join(test_path, subfolder)
    num_images = len(os.listdir(folder_path))
    print(f"{subfolder}: {num_images} images")


Number of images in each test folder:
0: 131 images
1: 131 images
2: 131 images
3: 131 images
4: 131 images
5: 131 images
6: 131 images
7: 131 images
8: 131 images
9: 131 images
add: 131 images
dec: 131 images
div: 131 images
eq: 131 images
mul: 131 images
sub: 131 images
x: 131 images
y: 131 images
z: 131 images
